In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)

from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

#configuting TPUs


In [10]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [11]:



train = pd.read_csv('/content/kaggle_data/jigsaw-toxic-comment-train.csv',engine='python')
validation = pd.read_csv('/content/kaggle_data/validation.csv')
test = pd.read_csv('/content/kaggle_data/test.csv')

In [13]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [14]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [15]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [16]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [17]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

#RNN Model

In [18]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [21]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
CPU times: user 252 ms, sys: 8.35 ms, total: 260 ms
Wall time: 302 ms


In [22]:
# model.fit(xtrain_pad, ytrain, epochs=2, batch_size=64*strategy.num_replicas_in_sync)
model.fit(xtrain_pad.astype(np.float32), ytrain.astype(np.float32), epochs=2, batch_size=64*strategy.num_replicas_in_sync) 

Epoch 1/2
150/150 [==============================] - 144s 944ms/step - loss: 0.2993 - accuracy: 0.9052
Epoch 2/2
150/150 [==============================] - 141s 938ms/step - loss: 0.1095 - accuracy: 0.9618


In [23]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.83%


In [24]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,yvalid)})

In [25]:
xtrain_seq[:1]

[[664,
  65,
  7,
  19,
  2262,
  14102,
  5,
  2262,
  20439,
  6071,
  4,
  71,
  32,
  20440,
  6620,
  39,
  6,
  664,
  65,
  11,
  8,
  20441,
  1502,
  38,
  6072]]

#word embeddings

In [26]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/content/kaggle_data/glove.6B.50d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:06, 60601.27it/s]

Found 400000 word vectors.


#LSTM s

In [27]:
#code explanaion

# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 647731.24it/s]


In [28]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     50,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 50)          2174850   
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2,235,351
Trainable params: 60,501
Non-trainable params: 2,174,850
_________________________________________________________________
CPU times: user 148 ms, sys: 5.28 ms, total: 153 ms
Wall time: 152 ms


In [31]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 350s 2s/step - loss: 0.1514 - accuracy: 0.9434
Epoch 2/5
150/150 [==============================] - 359s 2s/step - loss: 0.1498 - accuracy: 0.9451
Epoch 3/5
150/150 [==============================] - 349s 2s/step - loss: 0.1434 - accuracy: 0.9473
Epoch 4/5
150/150 [==============================] - 349s 2s/step - loss: 0.1351 - accuracy: 0.9510
Epoch 5/5
150/150 [==============================] - 352s 2s/step - loss: 0.1310 - accuracy: 0.9496


In [32]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.97%


In [33]:
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores,yvalid)})

#GRU s


In [34]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     50,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1500, 50)          2174850   
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 50)         0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 300)               315900    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 2,491,051
Trainable params: 316,201
Non-trainable params: 2,174,850
_________________________________________________________________
CPU times: user 153 ms, sys: 0 ns, total: 153 ms
Wall time: 152 ms


In [35]:
model.fit(xtrain_pad, ytrain, epochs=2, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/2
150/150 [==============================] - 243s 2s/step - loss: 0.2554 - accuracy: 0.9128
Epoch 2/2
150/150 [==============================] - 246s 2s/step - loss: 0.1900 - accuracy: 0.9297


In [36]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.95%


In [37]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores,yvalid)})

In [38]:
scores_model

[{'AUC_Score': 0.8297500699090978, 'Model': 'SimpleRNN'},
 {'AUC_Score': 0.9716452751581566, 'Model': 'LSTM'},
 {'AUC_Score': 0.952911663269152, 'Model': 'GRU'}]

#bidirectional RNNS

In [39]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     50,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1500, 50)          2174850   
                                                                 
 bidirectional (Bidirectiona  (None, 600)              842400    
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 601       
                                                                 
Total params: 3,017,851
Trainable params: 843,001
Non-trainable params: 2,174,850
_________________________________________________________________
CPU times: user 517 ms, sys: 0 ns, total: 517 ms
Wall time: 512 ms


In [40]:
model.fit(xtrain_pad, ytrain, epochs=2, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/2
150/150 [==============================] - 843s 6s/step - loss: 0.2525 - accuracy: 0.9136
Epoch 2/2
150/150 [==============================] - 848s 6s/step - loss: 0.1992 - accuracy: 0.9280


In [41]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.93%


In [42]:
scores_model.append({'Model': 'Bi-directional LSTM','AUC_Score': roc_auc(scores,yvalid)})

#Visualizations

In [43]:
# Visualization of Results obtained from various Deep learning models
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

,Model,AUC_Score
1,LSTM,0.971645
2,GRU,0.952912
3,Bi-directional LSTM,0.929763
0,SimpleRNN,0.829750


In [44]:
fig = go.Figure(go.Funnelarea(
    text =results.Model,
    values = results.AUC_Score,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()